In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
midwest_data = pd.read_csv('MidwestMurderData.csv')
midwest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145079 entries, 0 to 145078
Data columns (total 35 columns):
197601001IL00103                         145079 non-null object
Adams                                    145079 non-null object
 IL                                      145079 non-null object
IL00103                                  145079 non-null object
Illinois                                 145079 non-null object
Quincy                                   145079 non-null object
Municipal police                         145079 non-null object
FBI                                      145079 non-null object
Yes                                      145079 non-null object
1976                                     145079 non-null int64
ILL                                      145079 non-null object
January                                  145079 non-null object
1                                        145079 non-null int64
Normal update                            145079 non-null ob

In [4]:
midwest_data.head(100)

,197601001IL00103,Adams,IL,IL00103,Illinois,Quincy,Municipal police,FBI,Yes,1976,...,Friend,Brawl due to influence of alcohol,NULL,0,1.1,30180,Illinois.1,Rural Illinois,NULL.1,NULL.2
0,197601001IL01000,Champaign,IL,IL01000,Illinois,Champaign County,Sheriff,FBI,No,1976,...,Relationship not determined,Narcotic drug laws,NaN,0,0,30180,Illinois,Champaign-Urbana,IL,NaN
1,197601001IL01600,Cook,IL,IL01600,Illinois,Cook County,Sheriff,FBI,Yes,1976,...,Stranger,Robbery,NaN,0,1,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
2,197601001IL01619,Cook,IL,IL01619,Illinois,Chicago Heights,Municipal police,FBI,Yes,1976,...,Acquaintance,Other arguments,NaN,0,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
3,197601001IL01621,Cook,IL,IL01621,Illinois,Cicero,Municipal police,FBI,Yes,1976,...,Other - known to victim,Other arguments,NaN,0,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
4,197601001IL01625,Cook,IL,IL01625,Illinois,Des Plaines,Municipal police,FBI,Yes,1976,...,Acquaintance,Brawl due to influence of alcohol,NaN,0,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
5,197601001IL01643,Cook,IL,IL01643,Illinois,Harvey,Municipal police,FBI,Yes,1976,...,Stranger,Robbery,NaN,1,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
6,197601001IL01643,Cook,IL,IL01643,Illinois,Harvey,Municipal police,FBI,Yes,1976,...,Stranger,Robbery,NaN,1,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
7,197601001IL01675,Cook,IL,IL01675,Illinois,Northbrook,Municipal police,FBI,No,1976,...,Relationship not determined,Circumstances undetermined,NaN,0,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
8,197601001IL01693,Cook,IL,IL01693,Illinois,Riverdale,Municipal police,FBI,Yes,1976,...,Brother,Other arguments,NaN,0,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
9,197601001IL02207,DuPage,IL,IL02207,Illinois,Elmhurst,Municipal police,FBI,No,1976,...,Relationship not determined,All suspected felony type,NaN,0,0,30180,Illinois,Chicago-Naperville-Joliet,IL-IN-WI,NaN
